In [17]:
!pip install python-dotenv
!pip install requests

In [20]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
from datetime import datetime
import time
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
load_dotenv('/content/drive/MyDrive/Big Data Engineering/2024/Project 2/Final/secrets.env')

api_key = os.getenv("TOMTOM_API_KEY")

if api_key is None:
    raise ValueError("API key not found. Make sure your .env file is correct and the key is set.")


def optimize_delivery_route(start_lat, start_lon, stop_lat, stop_lon, api_key):
    url = f"https://api.tomtom.com/routing/1/calculateRoute/{start_lat},{start_lon}:{stop_lat},{stop_lon}/json"

    params = {
        'key': api_key,
        'traffic': 'true',           # Enable traffic data
        'routeType': 'fastest',      # Fastest route considering traffic
        'travelMode': 'car'
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        route_summary = data['routes'][0]['summary']

        # Feature Extraction
        route_data = {
            'timestamp': datetime.now(),
            'distance_km': route_summary['lengthInMeters'] / 1000,
            'travel_time_min': route_summary['travelTimeInSeconds'] / 60,
            'traffic_delay_min': route_summary['trafficDelayInSeconds'] / 60
        }

        return route_data
    else:
        print(f"Failed to get route: {response.status_code} - {response.text}")
        return None

# Real-time data processing loop with feature engineering
def process_route_data(start_lat, start_lon, stop_lat, stop_lon, api_key, interval=300, iterations=5):
    # Data collection for real-time processing
    route_data_records = []

    for _ in range(iterations):
        route_data = optimize_delivery_route(start_lat, start_lon, stop_lat, stop_lon, api_key)

        if route_data:
            # Append data to the records list
            route_data_records.append(route_data)


            df = pd.DataFrame(route_data_records)

            # Feature Engineering: Create new columns based on traffic patterns
            df['is_high_traffic'] = df['traffic_delay_min'].apply(lambda x: 1 if x > 10 else 0)
            df['efficiency_ratio'] = df['travel_time_min'] / df['distance_km']  # Travel time per km

            # Timestamp-based features
            df['hour_of_day'] = df['timestamp'].dt.hour
            df['day_of_week'] = df['timestamp'].dt.dayofweek


            return df


            df.to_csv('real_time_route_data.csv', index=False)

        # Wait before fetching the next set of data
        time.sleep(interval)


start_lat, start_lon = 33.9428, -118.4085  # LAX airport coordinates
stop_lat, stop_lon = 35.3035, -119.1534    #  Wine Cellars coordinates
api_key = api_key

process_route_data(start_lat, start_lon, stop_lat, stop_lon, api_key)


,timestamp,distance_km,travel_time_min,traffic_delay_min,is_high_traffic,efficiency_ratio,hour_of_day,day_of_week
0,2024-11-10 10:12:25.775547,193.488,114.416667,0.0,0,0.591337,10,6
